In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import numpy as np

import warnings
warnings.simplefilter('ignore')

<b>Домашнее задание:</b>  

Реализовать адаптивный бустинг использующий Логистическую Регрессию и меру ошибок LogLoss. Сравнить с точностью адаптивного бустинга на деревьях решений. Для сбора предсказаний можно использовать ту же функцию predict что и для бустинга на деревьях<br>  

<i>Примечание: в LogLoss необходимо передавать не предсказания полученные с помощью clf.predict(...), а вероятность, полученную с помощью clf.predict_proba(...)[:, 1]

In [2]:
class AdaBoost:
    def __init__(self, model_type='decision_tree'):
        self.models = []
        self.model_type = model_type
        self.n_classes = 0

    @staticmethod
    def log_loss(pred, y):
        return -np.sum(y * np.log2(pred) + (1 - y) * np.log2(1 - pred)) / len(y)

    @staticmethod
    def get_error(pred, y):
        return sum(pred != y) / len(y)

    def fit(self, X, y, n_models=50):
        # Запишем количество классов в переменную
        self.n_classes = len(np.unique(y))
        # Размер выборки
        n_objects = len(X)

        # Начальные веса деревьев
        w = np.ones(n_objects) / n_objects

        for n in range(n_models):
            # Реализация на решающих пнях
            if self.model_type == 'decision_tree':
                clf = DecisionTreeClassifier(max_depth=1)
                clf.fit(X, y, sample_weight=w)
                predictions = clf.predict(X)
                e = self.get_error(predictions, y)
            # на логистической регрессии
            elif self.model_type == 'logistic_regression':
                clf = LogisticRegression(max_iter=1000)
                clf.fit(X, y, sample_weight=w)
                predictions = clf.predict_proba(X)[:, 1]
                e = self.log_loss(predictions, y)
            else:
                raise ValueError('Wrong model type')

            # отбросим дерево, если его ошибка больше 0.5
            # Запишем условие в общем виде (применимо к небинарным классификаторам)
            if e >= 1 - 1 / self.n_classes:
                break

            # Вычислим вес для дерева
            alpha = 0.5 * np.log((1 - e) / e)

            # Найдем индексы правильно классифицированных элементов
            match = predictions == y

            # Увеличим веса для неправильно классифицированных элементов
            w[~match] *= np.exp(alpha)

            # Нормализуем веса
            w /= w.sum()

            # Добавим дерево с весом в список
            self.models.append((alpha, clf))

    def predict(self, X):
        n_objects = len(X)

        # вначале обозначим предсказание нулевым массивом
        y_pred = np.zeros((n_objects, self.n_classes))

        for alpha, clf in self.models:
            prediction = clf.predict(X)
            # Для каждого предсказания будем прибавлять alpha к
            # элементу с индексом предсказанного класса
            y_pred[range(n_objects), prediction] += alpha

        # выберем индексы с максимальными суммарными весами -
        # получим предсказанные алгоритмом классы
        y_pred = np.argmax(y_pred, axis=1)

        return y_pred

    def dec_predict(self, X):
        n_objects = len(X)
        score_bust = np.zeros(n_objects)

        for alpha, clf in self.models:
            score_i = clf.decision_function(X)
            score_bust += score_i * alpha
        y_pred = np.where(score_bust > 0, 1, 0)
        return y_pred

    def accuracy(self, X, y, decision=False):
        if decision:
            return (1 - self.get_error(self.dec_predict(X), y)) * 100
        else:
            return (1 - self.get_error(self.predict(X), y)) * 100

    def return_error(self, X, y, decision=False):
        if decision:
            return self.get_error(self.dec_predict(X), y)
        else:
            return self.get_error(self.predict(X), y)

In [3]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22)

На пнях

In [4]:
adaboost = AdaBoost(model_type='decision_tree')
adaboost.fit(X_train, y_train)
print(f'Точность алгоритма на обучающей выборке: {adaboost.accuracy(X_train, y_train):.3f}')
print(f'Точность алгоритма на тестовой выборке: {adaboost.accuracy(X_test, y_test):.3f}')

Точность алгоритма на обучающей выборке: 99.061
Точность алгоритма на тестовой выборке: 94.406


На логистической регрессии

In [5]:
adaboost = AdaBoost(model_type='logistic_regression')
adaboost.fit(X_train, y_train)
print(f'Точность алгоритма на обучающей выборке: {adaboost.accuracy(X_train, y_train):.3f}')
print(f'Точность алгоритма на тестовой выборке: {adaboost.accuracy(X_test, y_test):.3f}')

Точность алгоритма на обучающей выборке: 94.836
Точность алгоритма на тестовой выборке: 93.706


<b>Домашнее задание(необязательное, повышенной сложности):</b>  

Реализовать специальную функцию predict для бустинга на логистической регрессии выводящую предсказания по формуле: $ Predictions=sign(Score_{bust}) $,
где sign равен единице для положительных и нулю для отрицательных значений, а $ Score_{bust}= \sum \alpha_iScore_i$. Баллы выдаваемые каждой моделью $Score_i$ можно найти при помощи вызова метода decision_function на моделе

In [6]:
adaboost = AdaBoost(model_type='logistic_regression')
adaboost.fit(X_train, y_train)
print(f'Точность алгоритма на обучающей выборке: {adaboost.accuracy(X_train, y_train, decision=True):.3f}')
print(f'Точность алгоритма на тестовой выборке: {adaboost.accuracy(X_test, y_test, decision=True):.3f}')

Точность алгоритма на обучающей выборке: 94.836
Точность алгоритма на тестовой выборке: 93.706
